In [ ]:
# base human game working through timer


### Code originally taken from:
# https://levelup.gitconnected.com/writing-tetris-in-python-2a16bddb5318
# highly modified
import pygame
import random
import sys
import figure
Figure = figure.Figure
# Global variables
# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
TRANSPARENCY = 50
size = (800, 800)
        
# colors order is linked to figure order in figure.py, this seems like a potential problem
colors = [
    (255, 255, 255), # white for empty
    (0, 255, 255), # I- Cyan
    (0, 0, 255), # J - Blue
    (255, 127, 0), # L - Orange
    (255, 255, 0), # O - Yellow
    (0, 255, 0), # S - Green 
    (128, 0, 128), # T - Purple
    (255, 0, 0), # Z - Red
    (255, 215, 0), # GOLD, not implemented
    (194, 189, 176), # SILVER, not implemented
]


class Tetris:
    buffer = 4 # how many rows above actual grid to birth tetrominos
    n_queue = 4 # how many next tetrominos to show coming up
    # coordinates of board top left on larger screen
    x = 200
    y = 100
    
    fps = 60 # frames per second
    height = 20
    width = 10
    
    lines_per_level = 10 # num lines to get to next level
    points_per_line = [100, 300, 500, 800] # multiplied by level
    # number of frames (fps=60) that each level plays at, not level 29+ is nearly impossible for humans so may need to adjust
    level_frames_per_drop = {0:48, 
                             1:43, 
                             2:38, 
                             3:33, 
                             4:28, 
                             5:23, 
                             6:18, 
                             7:13, 
                             8:8, 
                             9:6, 
                             10:5, 11:5, 12: 5,
                             13:4, 14:4, 15:4,
                             16:3, 17:3, 18:3,
                             19:2, 20:2, 21:2, 22:2, 23:2, 24:2, 25:2, 26:2, 27:2, 28:2,
                             29:1}
        
    max_level = 29   
        
    # tetromino starting coordinates with buffer, 4 right, 2 down
    tet_x = 4
    tet_y = 2
    
    # coordinates of swap piece with respect to top left of SCREEN
    swap_x = 50
    swap_y = 200
    
    # Queue coordinates top piece
    queue_x = 500
    queue_y = 250
    zoom = 25 # size of grid squares
    
    change_player = False # used to make sure play switch happens only once when button is pressed
    
    
    # BASICALLY RESET FUNCTION, should that be a separate function?
    def __init__(self, player = 0, render= True ):
        self.game_start_time = pygame.time.get_ticks() # game start time
        self.player = player # 0 human, 1 computer
        self.full_height = self.height + self.buffer # including buffer region above
        
        self.landed_blocks = 0 # a metric for reward, small score for landing blocks
        self.board = [] # the main part of the state space (the grid of blocks)
        self.score = 0 
        self.lines = 0
        self.level = 0 # used for speed
        self.frames_per_drop = self.level_frames_per_drop[self.level]
        self.state = "start" # "start" and "gameover" are options
        
        # Only allow not showing game if computer is playing for training
        if player == 0:
            self.render = True
        else:
            self.render = render
        self.queue = [] # contains next pieces
        self.has_swapped = False # to allow user to save a tetromino to swap for later, once per drop
        self.swap_piece = None 
        self.total_reward = 0 # Used for training, always displayed
        
        # Set waiting queue and current figure
        self.figure =  Figure(x=self.tet_x,  y=self.tet_y) # piece currently dropping
        for i in range(self.n_queue):
            self.queue.append(Figure(x=self.tet_x,  y=self.tet_y))

        # Set board to be all 0's, including buffer region above
        for i in range(self.full_height): # add rows above game matrix
            new_line = []
            for j in range(self.width):
                new_line.append(0)
                
                
                
            # TESTING KICKS, this example: https://tetris.fandom.com/wiki/SRS:
                # if (i,j) in [(23,0),(23,1), (23,2), (23,3), (23,4), (23,6), (23,7), (23,8), (23,9),
                #              (22,0),(22,1), (22,2), (22,3), (22,6), (22,7), (22,8), (22,9),
                #              (21,0),(21,1), (21,6), (21,7), (21,8), (21,9),
                #              (20,1), (20,2), (20,3), (20,7), (20,8), (20,9),
                #              (19,6), (19,7), (19,8), (19,9),
                #              (18,6), (18,7), (18,5),
                #              (17,4), (17,5)]:
                #     new_line.append(1)
                # else:
                #     new_line.append(0)
                    
                    
                    
                    
            self.board.append(new_line)

    # called in init, when blocks freeze/line break, and debug call for chosen tetromino
    def new_figure(self, mode=None):
        # if short on blocks, add them to queue, pop the first one from the list, add another to end
        while len(self.queue) < self.n_queue:
            self.queue.append(Figure(x=self.tet_x,  y=self.tet_y))
        if mode is None:
            self.figure = self.queue.pop(0)
            self.queue.append(Figure(x=self.tet_x,  y=self.tet_y))
        else:
            self.figure = Figure(x=self.tet_x,  y=self.tet_y, mode=mode+1) # plus 1 to fix indexing
        

    def intersects(self):
        intersection = False
        for ind in self.figure.image():
            i = ind//4
            j = ind%4
            # check was > 0, but I want negative numbers to eventually check if block has been broken for gold/silver squares
            if i + self.figure.y >= self.full_height or \
                    j + self.figure.x >= self.width or \
                    j + self.figure.x < 0 or \
                    self.board[i + self.figure.y][j + self.figure.x] != 0: 
                intersection = True
        return intersection

    def break_lines(self):
        lines = 0
        # original code from https://levelup.gitconnected.com/writing-tetris-in-python-2a16bddb5318
        for i in range(self.buffer, self.full_height):
            zeros = 0
            for j in range(self.width):
                if self.board[i][j] == 0:
                    zeros += 1
                    break
                # else:
                #     zeros = 1# random number not 0
                #     break
            if zeros == 0:
                lines += 1
                self.lines += 1
                if self.lines % 10 == 0:
                    self.level += 1
                    
                    
                    # POTENTIAL ISSUE IF COUNTER HITS RIGHT MODULO AND DROPS BLOCK IMMEDIATELY HERE
                    self.frames_per_drop = self.level_frames_per_drop[min(self.level, self.max_level)]
                    # udpate frames per drop variable next
                # CHECK RANGE BOUNDARIES, seems to stop at second row 
                # at top
                for i1 in range(i, 1, -1):
                    for j in range(self.width):
                        self.board[i1][j] = self.board[i1 - 1][j]
       
    
        # TO DO: FIX SCORE FUNCTION
        #  Include Gold/Silver blocks
        if lines > 0:
            self.score += self.points_per_line[lines-1] * (self.level+1) # lines - 1 and self.level + 1 because of 0 indexing

        
    # returns the y value of where piece will drop, used for drawing shadow block
    def shadow_height(self): 
        old_y = self.figure.y    
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        new_y = self.figure.y
        self.figure.y = old_y
        return new_y
        
    def go_space(self):
        if self.state == "gameover":
            return
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

    def go_down(self):
        self.figure.y += 1
        if self.intersects():
            self.figure.y -= 1
            self.freeze()

    def freeze(self):
        # for i in range(4):
        #     for j in range(4):
        for ind in self.figure.image():
            i = ind//4
            j = ind%4
                # if i * 4 + j in self.figure.image():
            self.board[i + self.figure.y][j + self.figure.x] = self.figure.type

        self.landed_blocks += 4
        self.break_lines()
        self.has_swapped = False
        
        
        # hopefully a better/quicker way to check for gameover
        for j in range(self.width):
            # check row = 3 (4th row), any non zero elements mean Game Over
            if self.board[self.buffer-1][j] != 0: # 
                self.state = "gameover"
                return # no need to plot next figure
                
        self.new_figure()
            

    def swap(self):
        if self.has_swapped:
            # SHOULD play error sound cause you can't swap when you already swapped
            return
        
        self.has_swapped = True
        # save current piece for later
        if not(self.swap_piece):
            self.swap_piece = Figure(x=self.tet_x,  y=self.tet_y, mode= self.figure.type)
            self.new_figure()
        else:
            temp_piece = Figure(x=self.tet_x,  y=self.tet_y, mode= self.swap_piece.type)
            self.swap_piece = Figure(x=self.tet_x,  y=self.tet_y, mode= self.figure.type)
            self.figure = temp_piece

        

    def go_side(self, dx):
        old_x = self.figure.x
        self.figure.x += dx
        if self.intersects():
            self.figure.x = old_x

    def rotate(self, direction):
        # no rotation for the O block
        if self.figure.name == 'O':
            return
        
        old_rotation = self.figure.rotation
        self.figure.rotate(direction)
        
        # BASE CASE, no kick needed
        if not(self.intersects()):
        #     self.figure.rotation = old_rotation
        # else: 
            return
        
        # Failed without nudge
        #   4 states for each block, not magic number
        new_rotation = (old_rotation + direction)%4 
        
        if self.figure.name == 'I':
            nudges = self.figure.kicks_I[(old_rotation, new_rotation)]
        else:
            nudges = self.figure.kicks_main[(old_rotation, new_rotation)]
        
        old_x = self.figure.x
        old_y = self.figure.y
        
        for nudge in nudges:
            self.figure.x += nudge[0]
            self.figure.y += nudge[1]
            if not (self.intersects()):
                return
            else:
                self.figure.x = old_x
                self.figure.y = old_y
        
        # IF no nudges work
        self.figure.rotation = old_rotation
        

        
# One random function- BETTER SPOT FOR THIS GUY?    
# Transparency for block shadow, code taken from: https://stackoverflow.com/questions/6339057/draw-a-transparent-rectangles-and-polygons-in-pygame
def draw_rect_alpha(surface, color, rect):
    shape_surf = pygame.Surface(pygame.Rect(rect).size, pygame.SRCALPHA)
    pygame.draw.rect(shape_surf, color, shape_surf.get_rect())
    surface.blit(shape_surf, rect)

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
# Initialize the game engine
pygame.init()
# set screen (size set at top 800x800


# GAME LOOP 
# Loop until the user clicks the close button.
done = False
clock = pygame.time.Clock()
game = Tetris(player = 0, render=True)
if game.render:
    screen = pygame.display.set_mode(size)
    pygame.display.set_caption("Tetris")

counter = 0
pressing_down = False
pressing_sideways = 0


cur_time = 0


























while not done:
    
    # create a new piece if one doesn't exist
    # if game.figure is None:
    #     game.new_figure()
        
    # Tetris property I THINK, this whole loop!
    counter += 1
    if counter > 100000:
        counter = 0


    # calculate reward
    # dividing by time doesn't work without rendering- points go extremely high
    game.reward = (game.score + game.landed_blocks) / ((game.landed_blocks+1)/4) # / ((pygame.time.get_ticks()-game.game_start_time)/1000) 
    
    # if game.state == 'gameover':
    #     print(f'Reward: {game.reward}')
        
    if game.player == 1 and game.state == 'start': # computer play
        # choose random action 
        actions = ['no_op', 'left', 'right', 'down', 'hard', 'cw', 'ccw', 'swap']
        act = random.sample(actions,1)[0]
        pressing_sideways = 0

        if act == actions[0]:
            pass
        elif act == actions[1]:
            game.go_side(-1)
        elif act == actions[2]:
            game.go_side(1)        
        elif act == actions[3]:
            game.go_down()
        # elif act == actions[4]:
        #     game.go_space()
        elif act == actions[5]:
            game.rotate(direction = 1)
        elif act == actions[6]:
            game.rotate(direction = -1)
        elif act == actions[7]:
            game.swap()
            
    # Every time counter gets to ~ 60//level//2 (or down key)
    # drop piece (TOTALLY WRONG! FIX, somehow works though)
    # if game.state == 'start' and (counter % (fps // game.level // 2) == 0 or pressing_down):
    #     game.go_down()
            
    
    if game.state == 'start' and (counter% game.frames_per_drop == 0 or pressing_down):
        game.go_down()
        counter = 0
    
    
            
            
    # TOO FAST
    if pressing_sideways != 0: 
        if game.player == 1 or counter%2 == 0: 
            game.go_side(pressing_sideways)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
                        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:  
                game.__init__(player = game.player)
                
            # Don't attempt a player change if not rendering, only for training
            if event.key == pygame.K_p and game.render == False:
                game.change_player = True
                
            if game.player == 1:
                break # should exit FOR loop if computer playing, don't take inputs
                
                
                
            if event.key == pygame.K_l:
                game.lines += 1
                if game.lines % game.lines_per_level == 0:
                    game.level += 1
                    game.frames_per_drop = game.level_frames_per_drop[min(game.level, game.max_level)]
                
            if game.state == 'gameover':
                pressing_down = False
                pressing_sideways = 0
                break
            if event.key == pygame.K_RSHIFT:
                game.rotate(direction = 1)
            if event.key == pygame.K_SLASH:
                game.rotate(direction = -1)
            if event.key == pygame.K_DOWN:
                pressing_down = True
            else:
                pressing_down = False  
            if event.key == pygame.K_LEFT:
                pressing_sideways = -1
            elif event.key == pygame.K_RIGHT:
                pressing_sideways = 1
            else:
                pressing_sideways = 0
            if event.key == pygame.K_SPACE:
                game.go_space()         
            if event.key in [pygame.K_0, pygame.K_1, pygame.K_2, 
                             pygame.K_3, pygame.K_4, pygame.K_5, pygame.K_6]:
                game.new_figure(mode = event.key - pygame.K_0) # ASSUME K_0 is 48 and rest of numbers go up by 1            
            if event.key == pygame.K_s:
                game.swap()          
    if event.type == pygame.KEYUP:
                
            if event.key == pygame.K_DOWN:
                pressing_down = False
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                pressing_sideways = 0
            if event.key == pygame.K_p and game.change_player: # change players
                game.player = (game.player + 1) % 2
                game.change_player = False

                
                
    if not(game.render):
        # Still want 60fps?
        clock.tick(game.fps)
        continue
    screen.fill(WHITE)

    # Drawing screen
    for i in range(game.buffer, game.full_height):
        for j in range(game.width):
            # WHAT IS ZOOM of 20 doing for the rectangle drawing
            pygame.draw.rect(screen, GRAY, [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], width = 3)
            if game.board[i][j] > 0 and i >= game.buffer:
                pygame.draw.rect(screen, colors[game.board[i][j]],
                                 [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2, game.zoom - 1])

    # UPDATE#
    if game.figure is not None:
        shadow_y = game.shadow_height()
        for ind in game.figure.image():
            i = ind//4
            j = ind%4
            p = i * 4 + j
            
            # Plotting of shadow piece
            if shadow_y + i >= game.buffer:
                draw_rect_alpha(screen, 
                                tuple(list(colors[game.figure.type])+[TRANSPARENCY]),
                                 [game.x + game.zoom * (j + game.figure.x) + 1,
                                  game.y + game.zoom * (i + shadow_y) + 1,
                                  game.zoom - 2, game.zoom - 2])

            # Plotting of actual piece
            if game.figure.y + i >= game.buffer:
                pygame.draw.rect(screen, colors[game.figure.type],
                                 [game.x + game.zoom * (j + game.figure.x) + 1,
                                  game.y + game.zoom * (i + game.figure.y) + 1,
                                  game.zoom - 2, game.zoom - 2])
                
                
    # Plot SWAP piece if it has been set aside
    if game.swap_piece:
        for ind in game.swap_piece.image():
            i = ind//4
            j = ind%4
            pygame.draw.rect(screen, colors[game.swap_piece.type],
                             [game.swap_x + game.zoom * j,
                              game.swap_y + game.zoom * i,
                              game.zoom - 2, game.zoom - 2]) 
    else:
        # draw something indicating what this spot is for
        pass
    
    fig_i = 0
    for fig in game.queue:
        for ind in fig.image():
            i = ind//4
            j = ind%4
            pygame.draw.rect(screen, colors[fig.type],
                             [game.queue_x + game.zoom * j,
                              game.queue_y + game.zoom * (i + fig_i*5) , # testing coordinates
                              game.zoom - 2, game.zoom - 2]) 

        fig_i += 1
       
    
    
    # Displaying screen text
    font = pygame.font.SysFont('Calibri', 25, True, False)
    font1 = pygame.font.SysFont('Calibri', 65, True, False)
    text_score = font.render("Score: " + str(game.score), True, BLACK)
    text_lines = font.render("Lines: " + str(game.lines), True, BLACK)
    text_level = font.render("Level: " + str(game.level), True, BLACK)
    text_game_over = font1.render("Game Over", True, (255, 125, 0))
    text_game_over1 = font1.render("Press q", True, (255, 215, 0))
    text_swap = font.render("SWAP!", True, BLACK)
    text_queue = font.render("Queue:", True, BLACK)
    text_reward = font.render(f'Reward: {round(game.reward,2)}', True, BLACK)
    
    
    
    if game.player == 0:
        p = 'Human'
    else:
        p = 'Computer'
    text_player = font.render(f'{p}: \'p\' to swap', True, (200, 50, 100))     
        
    controlsX = 10
    controlsY = 300
    position = controlsX, controlsY
    font = pygame.font.SysFont('Calibri', 15)
    text_control = ["Controls",
            "/: CCW rotation",
           "rShift': CW rotation",
           "up,down,left,right: movement",
           "space: hard drop",
           "0-6: debug blocks",
           "s: swap",
           "q: restart game",
           "l: free line"]
    label = []
    for line in text_control: 
        label.append(font.render(line, True, GRAY))    
    for line in range(len(label)):
        screen.blit(label[line],(position[0],position[1]+(line*15)+(15*line)))

    screen.blit(text_player, [400, 50])
    screen.blit(text_score, [100, 50])
    screen.blit(text_lines, [100, 100])
    screen.blit(text_level, [100, 150])

    screen.blit(text_swap, [50, 250])
    screen.blit(text_queue, [game.queue_x, game.queue_y-50])
    screen.blit(text_reward, [0,0])
    # screen.blit(text_controls, [50, 300])
    if game.state == "gameover":
        screen.blit(text_game_over, [250, 80])
        screen.blit(text_game_over1, [250, 140])
    else:
        # update time if game is still going
        seconds=(pygame.time.get_ticks()-game.game_start_time)/1000
        text_timer = font.render(f'Time: {round(seconds)} s', True, BLACK)
        screen.blit(text_timer, [10, 50])
        
        
        
#         start_ticks=pygame.time.get_ticks() #starter tick
#      #calculate how many seconds
#     if seconds>10: # if more than 10 seconds close the game
#         break
#     print (seconds) #print how many seconds

    pygame.display.flip()
    clock.tick(game.fps)
pygame.quit()
